In [4]:
import numpy as np
from scipy.sparse import coo_matrix, csr_matrix

k = 5


def func_g(x, y):
    return np.sin(3 * x + 4 * y)


def generate_A(N):
    h = 1 / N
    size = (N - 1) ** 2
    const = h ** 2 * k ** 2
    A = np.zeros((size, size))
    for i in range(size):
        for j in range(i, size):
            if i == j:
                A[i, j] = A[j, i] = (24 - 4 * const) / 9
            elif (i == j - 1 and j % (N - 1) != 0) or j - i == N - 1:
                A[i, j] = A[j, i] = (-3 - const) / 9
            elif (j - i == N and j % (N - 1) != 0) or (j - i == N - 2 and i % (N - 1) != 0):
                A[i, j] = A[j, i] = (-12 - const) / 36

        # b =
        # (12 + const) / 36 * func_g(0, 0) + (3 + const) / 9 * (func_g(h, 0) + func_g(0, h))
        #  (12 + const) / 36 * func_g(1, 0) + (3 + const) / 9 * (func_g(1-h, 0) + func_g(1, h))
        # (12 + const) / 36 * func_g(0, 1) + (3 + const) / 9 * (func_g(h, 1) + func_g(0, 1-h))
        # (12 + const) / 36 * func_g(1, 1) + (3 + const) / 9 * (func_g(1-h, 1) + func_g(1, 1-h))

    print(A)
    return A


def generate_b(N):
    h = 1 / N
    size = (N - 1) ** 2
    nnz = 4 * (N - 2)
    row = np.array(nnz, dtype=np.int32)
    data = np.array(nnz)

    const = h ** 2 * k ** 2
    const1 = (12 + const) / 36
    const2 = (3 + const) / 9

    count = 0
    for condition in range(4):
        for i in range(N - 2):
            g1 = 0
            position = i + 1
            if condition == 0:
                if position == 1:
                    g1 = func_g(0, 0)
                    g2 = func_g(h, 0) + func_g(0, h)
                else:
                    g2 = func_g(position * h, 0)
                row[count] = position - 1
                data[count] = g1 * const1 + g2 * const2
                count += 1
            elif condition == 1:
                if position == 1:
                    g1 = func_g(1, 0)
                    g2 = func_g(1 - h, 0) + func_g(1, h)
                else:
                    g2 = func_g(1, position * h)
                row[count] = position * (N - 1) - 1
                data[count] = g1 * const1 + g2 * const2
                count += 1
            elif condition == 2:
                if position == 1:
                    g1 = func_g(1, 1)
                    g2 = func_g(1 - h, 1) + func_g(1, 1 - h)
                else:
                    g2 = func_g(position * h, 1)
                row[count] = (N - 1) ** 2 - position
                data[count] = g1 * const1 + g2 * const2
                count += 1
            elif condition == 3:
                position += 1
                if position == N - 1:
                    g1 = func_g(0, 1)
                    g2 = func_g(h, 1) + func_g(0, 1 - h)
                else:
                    g2 = func_g(0, position * h)
                row[count] = (N - 1) * (position - 1)
                data[count] = g1 * const1 + g2 * const2

    return coo_matrix(data, row, shape=nnz)


b = generate_b(3)